In [1]:
import numpy as np
from numpy import array
from sympy import symbols,  cos, sin, pi, simplify, sqrt, atan2, acos, Transpose
from sympy.matrices import Matrix

In [2]:
# Define DH param symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # theta_1
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') # d paramater
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')


# Joint angle symbols
# theta1, theta2, theta3, theta4, theta5, theta6 = 0


# Modified DH params
dh_params = {alpha0:        0, a0:      0, d1:  0.75,
             alpha1: -np.pi/2, a1:   0.35, d2:     0, q2: q2-np.pi/2,
             alpha2:        0, a2:   1.25, d3:     0,
             alpha3: -np.pi/2, a3: -0.054, d4:  1.50,
             alpha4:  np.pi/2, a4:      0, d5:     0,
             alpha5: -np.pi/2, a5:      0, d6:     0,
             alpha6:        0, a6:      0, d7: 0.303, q7: 0}

# General rotations around axes

R_z = Matrix([[       cos(q1),    -sin(q1),             0, 0],
              [       sin(q1),     cos(q1),             0, 0],
              [             0,           0,             1, 0],
              [             0,           0,             0, 1]])
R_y = Matrix([[       cos(q1),           0,       sin(q1), 0],
              [             0,           1,             0, 0],
              [      -sin(q1),           0,       cos(q1), 0],
              [             0,           0,             0, 1]])
R_x = Matrix([[             1,           0,             0, 0],
              [             0,     cos(q1),      -sin(q1), 0],
              [             0,     sin(q1),       cos(q1), 0],
              [             0,           0,             0, 1]])

# Define Modified DH Transformation matrix
R_corr = simplify(R_z.evalf(subs={q1: np.pi}) * R_y.evalf(subs={q1: -np.pi/2}))


In [3]:
# To rotate axis with yaw pitch roll, must do extrinsic XYZ rotation
# rotated_Matrix = Z[yaw] * Y[pitch] * X[roll] * original_Matrix 
alpha = symbols('alpha')
beta = symbols('beta')
gamma = symbols('gamma')
r11 = cos(alpha) * cos(beta)
r12 = (cos(alpha)* sin(beta) * sin(gamma)) - (sin(alpha)*cos(gamma))
r13 = (cos(alpha) * sin(beta) * cos(gamma)) + (sin(alpha)*sin(gamma))
r21 = sin(alpha) * cos(beta) * 1.0
r22 = (sin(alpha) * sin(beta) * sin(gamma)) + (cos(alpha) * cos(gamma))
r23 = (sin(alpha)*sin(beta)*cos(gamma)) - (cos(alpha)*sin(gamma))
r31 = -sin(beta)
r32 = cos(beta) * sin(gamma)
r33 = cos(beta)*cos(gamma)
extrinsic_rot_Matrix = Matrix([[r11, r12, r13],
                               [r21, r22, r23],
                               [r31, r32, r33]])


In [4]:
test_cases = {
    1: {
        "px": 2.153,
        "py": 0.0,
        "pz": 1.946,
        "roll": 0.0,
        "pitch": 0.0,
        "yaw": 0.0
    },
}

In [6]:
"""
To find the wrist center, one must use the roll, pitch, and yaw taken from the URDF
defined gripper and transform it into the DH param version. This is done by reversing
the R_corr (take the inverse). Probably faster to hard code the matrix though than
calculate the inverse everytime

Once R0_6 is found wrt DH params, we can use the n vector in the matrix to find
the wrist

WE DID IT!!!
"""
def find_wrist_center(px, py, pz, roll, pitch, yaw):
    end_effector_length = 0.303
    R0_6 = extrinsic_rot_Matrix.evalf(subs={alpha: roll, beta: pitch, gamma: yaw})
    # undo rotation to URDF orientation, back to DH params
    R0_6 = R0_6 * R_corr[0:3, 0:3].inv()
    print("extrinsic rotated")
    print(extrinsic_rot_Matrix * R_corr[0:3, 0:3].inv())[:, 2]
    
    wx = px - (end_effector_length) * R0_6[0, 2]
    wy = py - (end_effector_length) * R0_6[1, 2]
    wz = pz - (end_effector_length) * R0_6[2, 2]
    print "Wrist center from Matrix: ", (wx, wy, wz)
    
    wx = px - (end_effector_length) * 1.0 * cos(yaw) * cos(pitch)
    wy = py - (end_effector_length) * 1.0 * sin(yaw) * cos(pitch)
    wz = pz - (end_effector_length) * -1.0 * sin(pitch)
    print "Wrist center from hard-coded: ", (wx, wy, wz)
    
find_wrist_center(test_cases[1]['px'], test_cases[1]['py'], test_cases[1]['pz'], test_cases[1]['roll'], test_cases[1]['pitch'], test_cases[1]['yaw'])

# Final version
def find_wrist_center(px, py, pz, roll, pitch, yaw):
    end_effector_length = 0.303
    wx = px - (end_effector_length) * 1.0 * cos(yaw) * cos(pitch)
    wy = py - (end_effector_length) * 1.0 * sin(yaw) * cos(pitch)
    wz = pz - (end_effector_length) * -1.0 * sin(pitch)
    return wx, wy, wz

# Since R0_6 = Rrpy
def get_R0_6(alpha, beta, gamma):
    r11 = cos(alpha) * cos(beta)
    r12 = (cos(alpha)* sin(beta) * sin(gamma)) - (sin(alpha)*cos(gamma))
    r13 = (cos(alpha) * sin(beta) * cos(gamma)) + (sin(alpha)*sin(gamma))
    r21 = sin(alpha) * cos(beta) * 1.0
    r22 = (sin(alpha) * sin(beta) * sin(gamma)) + (cos(alpha) * cos(gamma))
    r23 = (sin(alpha)*sin(beta)*cos(gamma)) - (cos(alpha)*sin(gamma))
    r31 = -sin(beta)
    r32 = cos(beta) * sin(gamma)
    r33 = cos(beta)*cos(gamma)
    return Matrix([[r11, r12, r13],
                   [r21, r22, r23],
                   [r31, r32, r33]])

extrinsic rotated
Matrix([
[6.12323399573677e-17*sin(alpha)*sin(gamma) + 6.12323399573677e-17*sin(beta)*cos(alpha)*cos(gamma) + 1.0*cos(alpha)*cos(beta)],
[6.12323399573677e-17*sin(alpha)*sin(beta)*cos(gamma) + 1.0*sin(alpha)*cos(beta) - 6.12323399573677e-17*sin(gamma)*cos(alpha)],
[                                                                  -1.0*sin(beta) + 6.12323399573677e-17*cos(beta)*cos(gamma)]])
Wrist center from Matrix:  (1.85000000000000, -2.19268014489138e-142, 1.94600000000000)
Wrist center from hard-coded:  (1.85000000000000, 0.0, 1.94600000000000)
